In [1]:
%load_ext literary.module

# Notebook Finder

In [2]:
import os
import sys
from importlib.machinery import FileFinder, PathFinder
from inspect import getclosurevars

The notebook loader needs to be installed by modifying the existing `FileFinder`. This is so that packages can be loaded with `__init__.ipynb` or `__init__.py` modules.

In [3]:
def extend_file_finder(*loader_details):
    """Inject a set of loaders into a list of path hooks

    :param path_hooks: list of path hooks
    :param loader_details: FileFinder loader details
    :return:
    """
    for i, hook in enumerate(sys.path_hooks):
        try:
            namespace = getclosurevars(hook)
        except TypeError as err:
            continue

        try:
            details = namespace.nonlocals["loader_details"]
        except KeyError as err:
            continue

        break
    else:
        raise ValueError

    sys.path_hooks[i] = FileFinder.path_hook(*details, *loader_details)

    # To fix cached path finders
    sys.path_importer_cache.clear()